In [ ]:
%pip install ultralytics
# import ultralytics
# ultralytics.checks()

# 3. Train


In [ ]:
!cp /content/drive/MyDrive/datasets.zip .

In [ ]:
# # !curl -L "https://app.roboflow.com/ds/RUnpLRdMQA?key=7caFrn1e04" > roboflow.zip; unzip roboflow.zip -d datasets; rm roboflow.zip
# !curl -L "https://app.roboflow.com/ds/iSVZML2qTC?key=IzTFcEHNTi" > roboflow.zip; unzip roboflow.zip -d datasets; rm roboflow.zip
!rm -rf datasets
# !gdown --id 1gGgrj2Fp6kEEGkmSQpWQ7qeMRcwwGE-P

!unzip datasets.zip


In [ ]:
%%writefile datatraffic.yaml

train: train/images
val: valid/images

nc: 7
names: ["IC", "SMD", "capacitor", "inductor", "other", "resistor", "transistor"]

Writing datatraffic.yaml


In [ ]:
#large model
!yolo train model=yolov8l.pt data=datatraffic.yaml epochs=150 shear=0.2 flipud=0.5 augment=True max_det=100 batch=8 #resume=true#patience=300 mixup=0.3

Ultralytics YOLOv8.2.71 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8l.pt, data=datatraffic.yaml, epochs=150, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=100, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True

In [ ]:
# !yolo train model=yolov8n.pt data=datatraffic.yaml epochs=200 degrees=0.2 shear=0.2 flipud=0.5 augment=True max_det=500 batch=16

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp /content/runs/detect/train2/weights/last.pt /content/drive/MyDrive/lastPCB.pt

In [ ]:
!cp /content/drive/MyDrive/OutputImages.zip .

In [ ]:
!unzip OutputImages.zip

In [ ]:
!mkdir /content/output_txt
!rm /content/output_txt/*

In [ ]:
from tqdm import tqdm
from glob import glob
import os

from ultralytics import YOLO

model = YOLO('/content/drive/MyDrive/lastPCB.pt')

In [ ]:
 # load a pretrained YOLOv8n detection model

import re
def extract_numbers(filename):
    # print(filename)
    filename_ = filename.split("/")[-1]
    match = re.match(r'pcb(\d+)_rec(\d+)\.jpg', filename_)
    if match:
        # print(match)
        return int(match.group(1)), int(match.group(2))
    return float('inf'), float('inf')  # Đảm bảo các tệp không phù hợp sẽ được xếp cuối


img_list = glob("/content/content/OutputImages/*.jpg")
img_list = sorted(img_list, key=extract_numbers)

for image_path in tqdm(img_list[150:250]):
    # image_path = os.path.join(image_folder, image_name)

    results = model(image_path, classes=[2,3,4,5,6], iou=0.5, max_det=500)

    output_file_path = os.path.join("/content/output_txt", image_path.split("/")[-1].split(".")[0] + ".txt")
    with open(output_file_path, 'w') as output_file:
        for row, row2 in zip(results[0].boxes.xywhn, results[0].boxes.cls):
            output_file.write(f"{int(row2)} {row[0]} {row[1]} {row[2]} {row[3]}\n")

In [ ]:

!rm output_txt.zip
!zip -r "output_txt.zip" output_txt